In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser

In [2]:
mars_data = {}

# Mars news

In [3]:
news_url = 'https://mars.nasa.gov/news/'
news_html = requests.get(news_url)
news_soup = BeautifulSoup(news_html.text)
news_response = news_soup.find_all('div', class_='image_and_description_container')

In [4]:
news_titles = []
news_p = []
for i in range(len(news_response)):
    news_titles.append(news_response[i].find_all('img', alt=True)[1].get('alt',''))
    news_p.append(news_response[i].find('div',class_='rollover_description_inner').text.strip())

In [36]:
news = []
for i in range(len(news_titles)):
    news.append({'title':news_titles[i],'p':news_p[i]})

mars_data['news'] = news

# Mars picture

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
picture_html = browser.html
picture_soup = BeautifulSoup(picture_html, 'html.parser')


browser.click_link_by_partial_text('FULL IMAGE')

In [7]:
part_url = picture_soup.find_all('a', class_='button fancybox')[0].get('data-fancybox-href')

In [8]:
image_url = 'https://www.jpl.nasa.gov' + part_url
image = {'image_url':image_url}

In [28]:
mars_data['image'] = image_url

# Mars weather

In [10]:
weather_url =  'https://twitter.com/marswxreport?lang=en'
weather_html = requests.get(weather_url)
weather_soup = BeautifulSoup(weather_html.text)

In [11]:
mars_weather_all = weather_soup.find_all('p', class_='TweetTextSize')

In [12]:
mars_weather = mars_weather_all[1].text.replace ('\n', ', ')

In [13]:
mars_weather = mars_weather[:-26]

In [29]:
mars_data['weather'] = mars_weather

# Mars facts

In [48]:
facts_url = 'https://space-facts.com/mars/'
facts_html = requests.get(facts_url)
facts_soup = BeautifulSoup(facts_html.text)

In [60]:
mars_facts_table = pd.read_html(facts_url)

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [61]:
mars_facts_table = mars_facts_table[0].rename(columns={0:'Mars profile', 1: ''})
mars_facts_table = mars_facts_table.set_index('Mars profile')

In [63]:
mars_facts =  mars_facts_table.to_html()

In [64]:
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n    </tr>\n    <tr>\n      <th>Mars profile</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  

In [65]:
mars_data['facts'] = mars_facts

# Mars hemispheres

In [18]:
hemisphere_image_url = [{'title':'Cerberus hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
{'title': 'Schiaparelli hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
{'title': 'Syrtis major hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
{'title': 'Valles Marineris hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [31]:
mars_data['hemisphere_img'] = hemisphere_image_url

In [32]:
mars_data

{'news': {'Why This Martian Full Moon Looks Like Candy': "For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.",
  'NASA Garners 7 Webby Award Nominations': 'Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.',
  "NASA's Opportunity Rover Mission on Mars Comes to End": "NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.",
  "NASA's InSight Places First Instrument on Mars": 'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.',
  'NASA Announces Landing Site for Mars 2020 Rover': 'After a five-year search, NASA has chosen Jezero Crater as the land

In [43]:
mars_data['facts'].replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    <